In [ ]:
import requests
import os
import time
import random
import logging

# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='telegram_sender.log')

# Optional: Also log to the console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logging.getLogger('').addHandler(console_handler)

# Replace 'YOUR_BOT_TOKEN' with your actual bot token
BOT_TOKEN = 'YOUR_BOT_TOKEN'
# Replace 'YOUR_CHAT_ID' with the ID of the chat where you want to send the files
CHAT_ID = 'YOUR_CHAT_ID'

def send_with_retry(filename, chat_id, bot_token, max_retries=3):
    """Sends a file to Telegram with random wait and retry on failure."""
    url = f"https://api.telegram.org/bot{bot_token}/sendDocument?chat_id={chat_id}"
    retries = 0
    while retries < max_retries:
        try:
            logging.info(f"Attempting to send {filename} (attempt {retries + 1}/{max_retries})...")
            with open(filename, 'rb') as file:
                files = {"document": file}
                res = requests.post(url, files=files)
                res.raise_for_status()  # Raise an exception for bad status codes
                logging.info(f"Successfully sent {filename}")
                return True
        except requests.exceptions.RequestException as e:
            logging.error(f"Failed to send {filename} (attempt {retries + 1}/{max_retries}): {e}")
            if retries < max_retries - 1:
                wait_time = random.randint(5, 15)  # Wait between 5 to 15 seconds
                logging.info(f"Waiting for {wait_time} seconds before retrying for {wait_time} seconds...")
                time.sleep(wait_time)
            retries += 1
        except FileNotFoundError:
            logging.error(f"Error: File not found - {filename}")
            return False
        except Exception as e:
            logging.error(f"An unexpected error occurred while sending {filename}: {e}")
            return False

    logging.error(f"Failed to send {filename} after {max_retries} attempts.")
    return False

if __name__ == "__main__":
    logging.info("Script started.")
    tmp = list(os.scandir('.'))
    for item in tmp:
        if item.is_file() and 'mp3' in item.name:
            logging.info(f"Found MP3 file: {item.name}")
            send_with_retry(item.name, CHAT_ID, BOT_TOKEN)
    logging.info("Script finished.")